# Prevalence of ongoing symptoms following coronavirus (COVID-19) infection in the UK

We scrape the Excel files [published by ONS here](https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/datasets/alldatarelatingtoprevalenceofongoingsymptomsfollowingcoronaviruscovid19infectionintheuk) and save them to `data/tidy_ons_longcovid.csv`


In [2]:
library(tidyverse)
library(readxl)
library(rvest)

html <- read_html("https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/datasets/alldatarelatingtoprevalenceofongoingsymptomsfollowingcoronaviruscovid19infectionintheuk")
uri_nodes <- html %>% html_nodes(".btn--primary")%>% html_attr('href')

df <- data.frame(uri=uri_nodes)%>%
  filter(grepl('peoplepopulationandcommunity', uri), T)

In [10]:
# format before 20220303 with the exception of 20210401
readOldFormat <- function(filename, tableName){
    
  desc <- read_excel(filename, sheet = tableName, range="A2:A2") 

  a <- 
    read_excel(filename, sheet = tableName, range = "A5:E37")%>%
    mutate(`Impact on daily activity` = "Activity not limited")%>%
    fill(Domain)

  b <- cbind(
    read_excel(filename, sheet = tableName, range = "A5:B37"),
    read_excel(filename, sheet = tableName, range = "I5:K37"))%>%
    mutate(`Impact on daily activity` = "Activity limited a lot")%>%
    fill(Domain)


  c <- cbind(
    read_excel(filename, sheet = tableName, range = "A5:B37"),
    read_excel(filename, sheet = tableName, range = "F5:H37"))%>%
    mutate(`Impact on daily activity` = "Activity limited a little")%>%
    fill(Domain)

  data <-
    rbind(a, b)%>%
    rbind(c)%>%
    mutate(info = colnames(desc)[[1]])%>%
    mutate(filename = filename)
    
  return(data)
}

In [4]:
# format after 20220303
readNewFormat <- function(filename, tableName){
    
  columns <- c("Impact on daily activity", 
               "Domain", 
               "Group", 
               "Estimate", 
               "Lower 95% confidence limit", 
               "Upper 95% confidence limit")
    
  skipRows <- ifelse(grepl("20220303", filename), 3, 4)

  desc <- read_excel(filename, sheet = tableName, range="A1:A1") 

  data <- read_excel(filename, sheet = tableName, skip = skipRows) %>%
    select(all_of(columns))%>%
    mutate(info = colnames(desc)[[1]])%>%
    mutate(filename = filename)

  return(data)
}

## Download and parse all files listed on the ONS page

In [43]:
df_all <- data.frame()

for(uri in df$uri){
    long_uri <- paste0("https://www.ons.gov.uk", uri)
    
    vec <- str_split(long_uri, "/")
    filename <- paste0("data/",  vec[[1]][length(vec[[1]])])
    
    if(!file.exists(filename)){
        download.file(long_uri, destfile = filename)
    }
    tableName <-  tryCatch({
        read_excel(filename, sheet = "Table 9", range="A1:A1") 
        "Table 9"
    },
    error=function(cond) {
        "Table_9"
    })    

    if(grepl("accessible", filename)) { 

        print(paste("Reading new format", filename))        
        data <- readNewFormat(filename, tableName)  
 
    }else if(!grepl("20210401.xlsx", filename)){   
        
        print(paste("Reading old format", filename))          
        data <- readOldFormat(filename, tableName)  
        
     }else{
        print(paste("Not using", filename))
        data <- data.frame()
    }

    df_all <- rbind(df_all, data)
}

[1] "Reading new format data/longcovid1920230330accessible.xlsx"
[1] "Reading new format data/longcovid1920230302accessible.xlsx"
[1] "Reading new format data/longcovid1920230202accessible.xlsx"
[1] "Reading new format data/longcovid1920230105accessible.xlsx"
[1] "Reading new format data/longcovid1920221201accessible.xlsx"
[1] "Reading new format data/longcovid1920221103accessible.xlsx"
[1] "Reading new format data/longcovid1920221006accessible2.xlsx"
[1] "Reading new format data/longcovid1920220901accessible.xlsx"
[1] "Reading new format data/ongoingsymptomsfollowingcovid1920220804accessible.xlsx"
[1] "Reading new format data/ongoingsymptomsfollowingcovid1920220707accessible.xlsx"
[1] "Reading new format data/ongoingsymptomsfollowingcovid1920220601accessible.xlsx"
[1] "Reading new format data/ongoingsymptomsfollowingcovid1920220506accessible.xlsx"
[1] "Reading new format data/ongoingsymptomsfollowingcovid1920220407accessibleupdated.xlsx"
[1] "Reading new format data/ongoingsymptomsfol

## Consolidate the inconsistent groups
The warning `NAs introduced by coercion` stems from censored data.

In [44]:
df_all <- 
  df_all%>%  
  mutate(`Week Ending Date` = gsub(".*ending ", "", info))%>% 
  mutate(`Week Ending Date` = as.Date(`Week Ending Date`, format = "%d %B %Y"))%>%  
  mutate(Estimate = as.integer(Estimate))%>%  
  mutate(`Lower 95% confidence limit` = as.integer(`Lower 95% confidence limit`))%>%  
  mutate(`Upper 95% confidence limit` = as.integer(`Upper 95% confidence limit`))%>%  
  mutate(`Impact on daily activity` = gsub("A little", "Activity limited a little", `Impact on daily activity`))%>%
  mutate(`Impact on daily activity` = gsub("A lot", "Activity limited a lot", `Impact on daily activity`))%>%
  mutate(`Impact on daily activity` = gsub("Not at all", "Activity not limited", `Impact on daily activity`))%>%
  mutate(Group = gsub("<12 weeks", "4 to <12 weeks", Group))%>%
  mutate(Group = gsub("≥52 weeks", "52+ weeks", Group))%>%
  mutate(Group = gsub(">= 52 weeks", "52+ weeks", Group))%>%
  mutate(Group = gsub(">=104 weeks", "104+ weeks", Group))%>%
  mutate(Group = gsub("≥70 years", "70+", Group))%>%
  mutate(Group = gsub(" years", "", Group))%>%
  mutate(Group = gsub("Men", "Male", Group))%>%
  mutate(Group = gsub("Women", "Female", Group))%>%
  mutate(Domain = gsub("NHS contact at time.*", 
                      "NHS contact at time of first (suspected) coronavirus infection", Domain))%>%
  mutate(midpoint_date =`Week Ending Date` - 14)%>%  
  select(-info)

df_all%>%
  write_csv("data/tidy_ons_longcovid.csv")

df_all%>%  
  filter(Group == "52+ weeks")%>%  
  filter(`Impact on daily activity` == "Activity not limited")%>%
  arrange(`Week Ending Date`)%>%
  tail

Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `Estimate = as.integer(Estimate)`.
Caused by warning:
! NAs introduced by coercion”
Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `Lower 95% confidence limit = as.integer(`Lower 95% confidence
  limit`)`.
Caused by warning:
! NAs introduced by coercion”
Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `Upper 95% confidence limit = as.integer(`Upper 95% confidence
  limit`)`.
Caused by warning:
! NAs introduced by coercion”


Impact on daily activity,Domain,Group,Estimate,Lower 95% confidence limit,Upper 95% confidence limit,filename,Week Ending Date,midpoint_date
<chr>,<chr>,<chr>,<int>,<int>,<int>,<chr>,<date>,<date>
Activity not limited,Duration since first (suspected) coronavirus infection,52+ weeks,122,110,134,data/ongoingsymptomsfollowingcovid1920211007.xlsx,2021-09-05,2021-08-22
Activity not limited,Duration since first (suspected) coronavirus infection,52+ weeks,122,110,134,data/ongoingsymptomsfollowingcovid1920211104.xlsx,2021-10-02,2021-09-18
Activity not limited,Duration since first (suspected) coronavirus infection,52+ weeks,138,126,149,data/ongoingsymptomsfollowingcovid1920211202.xlsx,2021-10-31,2021-10-17
Activity not limited,Duration since first (suspected) coronavirus infection,52+ weeks,166,153,179,data/ongoingsymptomsfollowingcovid1920220106.xlsx,2021-12-06,2021-11-22
Activity not limited,Duration since first (suspected) coronavirus infection,52+ weeks,179,165,194,data/ongoingsymptomsfollowingcovid1920220203.xlsx,2022-01-02,2021-12-19
Activity not limited,Duration since first (suspected) coronavirus infection,52+ weeks,215,201,229,data/ongoingsymptomsfollowingcovid1920220303accessibleupdated.xlsx,2022-01-31,2022-01-17
